In [50]:
import os
import seaborn as sns
import matplotlib.pyplot as plt

import SALib.analyze.morris
from SALib.sample import morris

import warnings
warnings.filterwarnings('ignore')

import sys
sys.path.append('../')

from tqdm import tqdm

from infrasim.optimise import *
from infrasim.utils import *

In [ ]:
#File paths
nodes = '../data/nextra/spatial/network/nodes.shp'
edges = '../data/nextra/spatial/network/edges.shp'
flows = '../data/nextra/nodal_flows/processed_flows_2030.csv'

timesteps   = 24
num_levels  = 100
N           = 100

# Set up problem for sensitivity analysis # name:[lower_bound,upper_bound]
params = {'coop_res_target'                 : [0,0.5],
          'jordan_to_westbank'              : [0,10**12],
          'jordan_to_israel'                : [0,10**12],
          'israel_to_westbank'              : [0,10**12],
          'israel_to_jordan'                : [0,10**12],
          'israel_to_gaza'                  : [0,10**12],
          'westbank_to_israel'              : [0,10**12],
          'westbank_to_jordan'              : [0,10**12],
          'self_sufficiency_factor'         : [0,0.9],
         }

problem = {'num_vars'    : len(params.keys()),
           'names'       : [i for i in params.keys()],
           'bounds'      : [params[i] for i in params.keys()]
          }

# create parameter values
param_values = morris.sample(problem,N=N,
                             num_levels=num_levels,
                             local_optimization=True)

# Run analysis with the specified parameter values
collected_caps_israel   = []
collected_caps_jordan   = []
collected_caps_westbank = []
collected_caps_gaza     = []

for param_set in tqdm(param_values,total=len(param_values)):

    model_run = nextra(nodes,edges,flows,
                       timesteps=timesteps,
                       energy_objective=True,
                       scenario='COO',
                       super_sink=False,
                       super_source=False,
                       # params
                       coo_res_factor=param_set[0],
                       jordan_to_westbank=param_set[1],
                       jordan_to_israel=param_set[2],
                       israel_to_westbank=param_set[3],
                       israel_to_jordan=param_set[4],
                       israel_to_gaza=param_set[5],
                       westbank_to_israel=param_set[6],
                       westbank_to_jordan=param_set[7],
                       self_sufficiency_factor=param_set[8],
                      )

    model_run.build()
    model_run.run(pprint=False)
    model_results = model_run.get_results()
    
    collected_caps_israel.append(caps.loc[caps.territory == 'Israel','value'].values[0])
    collected_caps_jordan.append(caps.loc[caps.territory == 'Jordan','value'].values[0])
    collected_caps_westbank.append(caps.loc[caps.territory == 'West Bank','value'].values[0])
    collected_caps_gaza.append(caps.loc[caps.territory == 'Gaza','value'].values[0])



  0%|                                                                                       | 1/1000 [00:00<10:09,  1.64it/s]

  0%|▏                                                                                      | 2/1000 [00:01<09:06,  1.83it/s]

  0%|▎                                                                                      | 3/1000 [00:01<08:47,  1.89it/s]

  0%|▎                                                                                      | 4/1000 [00:02<09:18,  1.78it/s]

  0%|▍                                                                                      | 5/1000 [00:02<08:54,  1.86it/s]

...

 55%|██████████████████████████████████████████████▉                                      | 552/1000 [04:43<03:42,  2.01it/s]

 55%|███████████████████████████████████████████████                                      | 553/1000 [04:44<03:41,  2.02it/s]

 55%|███████████████████████████████████████████████                                      | 554/1000 [04:44<03:39,  2.03it/s]

 56%|███████████████████████████████████████████████▏                                     | 555/1000 [04:45<03:37,  2.04it/s]

 56%|███████████████████████████████████████████████▎                                     | 556/1000 [04:45<03:47,  1.96it/s]

In [75]:
def process_morris_results(problem,param_values,collected_results,num_levels):
    '''Process output from Morris sensitivitys
    '''
    Si = SALib.analyze.morris.analyze(problem,
                                      np.array(param_values),
                                      np.array(collected_results),
                                      print_to_console=False,
                                      num_levels=num_levels)

    sensitivity_results = pd.DataFrame.from_dict(Si)
    return sensitivity_results

israel_results = process_morris_results(problem,param_values,collected_caps_israel,num_levels)
jordan_results = process_morris_results(problem,param_values,collected_caps_jordan,num_levels)
westbank_results = process_morris_results(problem,param_values,collected_caps_westbank,num_levels)
gaza_results = process_morris_results(problem,param_values,collected_caps_gaza,num_levels)

array([[0.00000000e+00, 4.44444444e+11, 7.77777778e+11, 6.66666667e+11,
        8.88888889e+11, 4.44444444e+11, 4.44444444e+11, 0.00000000e+00,
        0.00000000e+00],
       [0.00000000e+00, 4.44444444e+11, 7.77777778e+11, 6.66666667e+11,
        8.88888889e+11, 4.44444444e+11, 4.44444444e+11, 5.55555556e+11,
        0.00000000e+00],
       [0.00000000e+00, 4.44444444e+11, 7.77777778e+11, 6.66666667e+11,
        8.88888889e+11, 1.00000000e+12, 4.44444444e+11, 5.55555556e+11,
        0.00000000e+00],
       [0.00000000e+00, 1.00000000e+12, 7.77777778e+11, 6.66666667e+11,
        8.88888889e+11, 1.00000000e+12, 4.44444444e+11, 5.55555556e+11,
        0.00000000e+00],
       [0.00000000e+00, 1.00000000e+12, 7.77777778e+11, 6.66666667e+11,
        8.88888889e+11, 1.00000000e+12, 1.00000000e+12, 5.55555556e+11,
        0.00000000e+00],
       [0.00000000e+00, 1.00000000e+12, 7.77777778e+11, 6.66666667e+11,
        3.33333333e+11, 1.00000000e+12, 1.00000000e+12, 5.55555556e+11,
        0.0